# Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
#print (keras.__version__)
#print (theano.__version__)
#print (tensorflow.__version__)

In [3]:
import pip
for m in pip.get_installed_distributions():
    if m.project_name == 'keras':
        print(m.version)
print ("end")

end


In [4]:
#set FIXED variables
HOMEPATH = "/home/ubuntu/fastai/"
VALIDPATH = HOMEPATH + 'data/state/valid/'
TRAINPATH = HOMEPATH + 'data/state/train/'
TESTPATH = HOMEPATH + 'data/state/test/'

SAMPLEPATH = HOMEPATH + 'data/state/sample/'
SAMPLEVALIDPATH = HOMEPATH + 'data/state/sample/valid/'
SAMPLETRAINPATH = HOMEPATH + 'data/state/sample/train/'
SAMPLETESTPATH = HOMEPATH + 'data/state/sample/test/'


import os, errno

from datetime import datetime
from shutil import copyfile, move
import random

In [5]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"
path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

('current working directory:', '/home/ubuntu/fastai')


Using TensorFlow backend.


In [6]:
batch_size=64

## Create sample

The following assumes you've already created your validation set - remember that the training and validation set should contain *different drivers*, as mentioned on the Kaggle competition page.

HOLD - markdown cell  
%cd data/state  
%cd train  
%mkdir ../sample  
%mkdir ../sample/train  
%mkdir ../sample/valid  


In [ ]:
#after unpacking files from kaggle data download. dirs and files already exist.

DIR = TESTPATH
print (DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
79726/10

In [ ]:
# dir and files already exist after unpacking kaggle data.
for i in range(0, 10):
    DIR = TRAINPATH+"c"+str(i)
    print (DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
#
def makeDir(directory):
    try:
        os.makedirs(directory)
        return 1
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
            return -1
        return 0
    

In [ ]:
#create dir for sample test
DIR = SAMPLETESTPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

for i in range(0, 10):
    DIR = SAMPLETESTPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
#create dir for validation
DIR = VALIDPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

for i in range(0, 10):
    DIR = VALIDPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
# move a % of the training data to validation directories.
validFract = 0.33
for i in range(0, 10):
    TRAINDIR = TRAINPATH+"c"+str(i)+"/"
    trainDirFiles = [name for name in os.listdir(TRAINDIR) if os.path.isfile(os.path.join(TRAINDIR, name))]
    numToMove = int(validFract*len(trainDirFiles))
    print (TRAINDIR, ":", len(trainDirFiles), numToMove, len(trainDirFiles)-numToMove)
    VALIDDIR = VALIDPATH+"c"+str(i)+"/"
    validDirFiles = [name for name in os.listdir(VALIDDIR) if os.path.isfile(os.path.join(VALIDDIR, name))]
    print (VALIDDIR, ":", len(validDirFiles))
    if len(validDirFiles)==0:
        #select a random validFract of files from TRAINDIR & move to VALIDDIR IFF not already done.
        moveList = random.sample(trainDirFiles, numToMove)
        print ("moving ", len(moveList))
        for file in moveList:
            #print(TRAINDIR+file, VALIDDIR+file)
            move(TRAINDIR+file, VALIDDIR+file)
            #break
    print()


In [ ]:
#create sample directories for training, validation and copy a % of files across.
#todo: fix overlap sampling error.
#select random files and copy to sample.
#select random files from sample and _MOVE_ to sample/train
#select random files from sample and MOVE to sample/valid
#move remaining files to sample/test  now have zero overlap between train/valid/test.

DIR = SAMPLEPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

DIR = SAMPLEVALIDPATH 
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

DIR = SAMPLETRAINPATH
print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))


for i in range(0, 10):
    DIR = SAMPLEVALIDPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))


for i in range(0, 10):
    DIR = SAMPLETRAINPATH+"c"+str(i)
    print (makeDir(DIR), DIR, ":", len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [ ]:
#NBBBB: __IMPORTANT___ this does not create test data correctly, potential overlap between test/train/validate.
# Need to segregate randomly selected data properly - maybe start again. UGLY algo below!!!!

sampleFract = 0.1

for i in range(0, 10):
    sampleTrainPath = SAMPLETRAINPATH+"c"+str(i)+"/"
    sampleTrainPathFiles = [name for name in os.listdir(sampleTrainPath) if os.path.isfile(os.path.join(sampleTrainPath, name))]
    print (sampleTrainPath, ":", len(sampleTrainPathFiles))
    sampleValidPath = SAMPLEVALIDPATH+"c"+str(i)+"/"
    sampleValidPathFiles = [name for name in os.listdir(sampleValidPath) if os.path.isfile(os.path.join(sampleValidPath, name))]
    print (sampleValidPath, ":", len(sampleValidPathFiles))
    sampleTestPath = SAMPLETESTPATH+"c"+str(i)+"/"
    sampleTestPathFiles = [name for name in os.listdir(sampleTestPath) if os.path.isfile(os.path.join(sampleTestPath, name))]
    print (sampleTestPath, ":", len(sampleTestPathFiles))
    
    TRAINDIR = TRAINPATH+"c"+str(i)+"/"
    trainDirFiles = [name for name in os.listdir(TRAINDIR) if os.path.isfile(os.path.join(TRAINDIR, name))]
    numToMoveTrain = int(sampleFract*len(trainDirFiles))
    print (TRAINDIR, ":", len(trainDirFiles), numToMoveTrain, len(trainDirFiles)-numToMoveTrain)

    VALIDDIR = VALIDPATH+"c"+str(i)+"/"
    validDirFiles = [name for name in os.listdir(VALIDDIR) if os.path.isfile(os.path.join(VALIDDIR, name))]
    numToMoveValid = int(sampleFract*len(validDirFiles))
    print (VALIDDIR, ":", len(validDirFiles), numToMoveValid, len(validDirFiles)-numToMoveValid)
    
    
    #now _COPY_ randomly selected files from train to sample/train    
    if len(sampleTrainPathFiles)==0:
        copyList = random.sample(trainDirFiles, numToMoveTrain)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(TRAINDIR+file, sampleTrainPath+file)
            copyfile(TRAINDIR+file, sampleTrainPath+file)
            #break
    print ()
    

    #now _COPY_ randomly selected files from valid to sample/valid 
    if len(sampleValidPathFiles)==0:
        copyList = random.sample(validDirFiles, numToMoveValid)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(VALIDDIR+file, sampleValidPath+file)
            copyfile(VALIDDIR+file, sampleValidPath+file)
            #break
    print ()
    #break
    


    #now _COPY_ randomly selected files from train to sample/test    
    if len(sampleTestPathFiles)==0:
        copyList = random.sample(trainDirFiles, numToMoveTrain)
        print ("copying ", len(copyList))
        for file in copyList:
            #print(TRAINDIR+file, sampleTestPath+file)
            copyfile(TRAINDIR+file, sampleTestPath+file)
            #break
    print ()

    


## Create batches

In [ ]:
batches = get_batches(SAMPLETRAINPATH, batch_size=batch_size)
val_batches = get_batches(SAMPLEVALIDPATH, batch_size=batch_size*2, shuffle=False)
print (SAMPLETRAINPATH, type(batches))
print (SAMPLEVALIDPATH, type(val_batches))

#https://keras.io/preprocessing/image/
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/DirectoryIterator


In [ ]:

(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(SAMPLEPATH)
print ("val_classes", type(val_classes), val_classes.shape)
print ("trn_classes", type(trn_classes), trn_classes.shape)
print ("val_labels", type(val_labels), val_labels.shape)
print ("val_labels", type(val_labels), val_labels.shape)
print ("trn_labels", type(trn_labels), trn_labels.shape)
print ("val_filenames", type(val_filenames), len(val_filenames))
print ("filenames", type(filenames), len(filenames))
print ("test_filename", type(test_filename), len(test_filename))


## Basic models

### Linear model

First, we try the simplest model and use default parameters. Note the trick of making the first layer a batchnorm layer - that way we don't have to worry about normalizing the input ourselves.  

simplest possible liner model  
- has single dense layer
- start with BatchNormalization layer. normalises the data for us. dont need to calculate the average, standard deviation etc.
- flatten layer required to convert images into a single vector first.

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
#NB: this above is most minimal and simplest model.
print (type(model))
print (len(model.layers))
for layer in model.layers:
    print (type(layer))

As you can see below, this training is going nowhere...

In [ ]:

startTime= datetime.now()
print ("model.compile start")
#https://keras.io/optimizers/#adam
#Adam: A Method for Stochastic Optimization
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=6, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)
#nb: original epoch was 2
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


video was 7seconds per epoch. mine is 119seconds. loss/accuracy/score is not improving over 2 epochs and low accuracy on both train and validation.

Let's first check the number of parameters to see that there's enough parameters to find some useful relationships:

In [ ]:
model.summary()

Over 1.5 million parameters - that should be enough. Incidentally, it's worth checking you understand why this is the number of parameters in this layer:

In [ ]:
10*3*224*224
#NB: 10*3*224*224 is not an exact match to the output from model.summary 1505280 vs 1505290

#low accuracy probably because learning rate is too high.

Since we have a simple model with no regularization and plenty of parameters, it seems most likely that our learning rate is too high. Perhaps it is jumping to a solution where it predicts one or two classes with high confidence, so that it can give a zero prediction to as many classes as possible - that's the best approach for a model that is no better than random, and there is likely to be where we would end up with a high learning rate. So let's check:

In [ ]:
print (type(batches))
#recall get_batches is from utils.py 
#uses gen.flow_from_directory(dirname, target_size=target_size,
#            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
#np.round(model.predict_generator(batches, batches.N)[:10],2)
np.round(model.predict_generator(batches, batches.nb_sample)[:10],2)

#first run : got 9 sixes and one 5.
#second run:  got 8 sevens and two 6's.

Our hypothesis was correct. It's nearly always predicting class 1 or 6, with very high confidence. So let's try a lower learning rate:

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
print (len(model.layers))
for layer in model.layers:
    print (type(layer))

In [ ]:
startTime= datetime.now()
print ("model.compile start")
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Great - we found our way out of that hole... Now we can increase the learning rate and see where we can get to.

In [ ]:
model.optimizer.lr=0.001

In [ ]:
startTime= datetime.now()
print ("model.fit_generator start")
model.fit_generator(batches, batches.nb_sample, nb_epoch=8, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)
#epoch was 4
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


We're stabilizing at validation accuracy of 0.39. Not great, but a lot better than random. Before moving on, let's check that our validation set on the sample is large enough that it gives consistent results:

In [ ]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)
#get_batches is from utlils
#uses  image.ImageDataGenerator.flow_from_directory 
#https://keras.io/preprocessing/image/
#Takes the path to a directory, and generates batches of augmented/normalized data. 
#Yields batches indefinitely, in an infinite loop.

How to evaluate if model size is large enough?  
use randomly sampled batches.
evaluate the range of accuracies resulting.

In [ ]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res, 2)

In [ ]:
print (type(val_res), len(val_res), type(val_res[0]), len(val_res[0]))

In [ ]:
#get max/min/std-dev of val_res
print ("min:", [round(elem, 2) for elem in min(val_res)] )
print ("max:", [round(elem, 2) for elem in max(val_res)] )
#print (np.array(val_res).shape)
print ("std:", np.std(np.array(val_res), axis=0))  
#print (std(val_res) )

Yup, pretty consistent - if we see improvements of 3% or more, it's probably not random, based on the above samples.

### L2 regularization

The previous model is over-fitting a lot, but we can't use dropout since we only have one layer. We can try to decrease overfitting in our model by adding [l2 regularization](http://www.kdnuggets.com/2015/04/preventing-overfitting-neural-networks.html/2) (i.e. add the sum of squares of the weights to our loss function):

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=l2(0.01))
    ])
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Looks like we can get a bit over 50% accuracy this way. This will be a good benchmark for our future models - if we can't beat 50%, then we're not even beating a linear model trained on a sample, so we'll know that's not a good approach.

### Single hidden layer

The next simplest model is to add a single hidden layer.

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

model.optimizer.lr = 0.01
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Not looking very encouraging... which isn't surprising since we know that CNNs are a much better choice for computer vision problems. So we'll try one.

### Single conv layer

video @ 1:04

2 conv layers with max pooling followed by a simple dense network is a good simple CNN to start with:

In [ ]:
def conv1(batches):
    '''
        
    '''
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),                    #1st convolutional layers
            BatchNormalization(axis=1),                                  #followed by BatchNormalization & MaxPooling
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),                    #2nd convolutional layers
            BatchNormalization(axis=1),                                  #followed by BatchNormalization & MaxPooling
            MaxPooling2D((3,3)), 
            Flatten(),                                                   #Flattens the input. Does not affect the batch size.
            Dense(200, activation='relu'),                               #Dense followed by BatchNormalization
            BatchNormalization(),
            Dense(10, activation='softmax')                              #NB: output layer is Dense w softmax
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [ ]:
conv1(batches)

noted from results above  
- training accuracy very quickly reaches 100%
- validation accuracy much lower (video was 24%, have 37% here)
- CLASSIC OVERFITTING!!!!!

The training set here is very rapidly reaching a very high accuracy. So if we could regularize this, perhaps we could get a reasonable result.

So, what kind of regularization should we try first? As we discussed in lesson 3, we should start with data augmentation.

## Data augmentation

To find the best data augmentation parameters, we can try each type of data augmentation, one at a time. For each type, we can try four very different levels of augmentation, and see which is the best. In the steps below we've only kept the single best result we found. We're using the CNN we defined above, since we have already observed it can model the data quickly and accurately.

Width shift: move the image left and right -

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Height shift: move the image up and down -

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Random shear angles (max in radians) -

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Rotation: max in degrees -

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

Channel shift: randomly changing the R,G,B colors - 

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

And finally, putting it all together!

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

In [ ]:
model = conv1(batches)

At first glance, this isn't looking encouraging, since the validation set is poor and getting worse. But the training set is getting better, and still has a long way to go in accuracy - so we should try annealing our learning rate and running more epochs, before we make a decisions.

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=5, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Lucky we tried that - we starting to make progress! Let's keep going.

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=25, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Amazingly, using nothing but a small sample, a simple (not pre-trained) model with no dropout, and data augmentation, we're getting results that would get us into the top 50% of the competition! This looks like a great foundation for our futher experiments.

To go further, we'll need to use the whole dataset, since dropout and data volumes are very related, so we can't tweak dropout without using all the data.

In [ ]:
model.count_params()

In [ ]:
model.summary()

In [ ]:
type(model)

In [ ]:
model.save_weights("Week4_statefarm-sample-working_saved_weights.h5")

In [ ]:
print (os.getcwd())


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("Week4_statefarm-sample-working.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
del loaded_model 

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('Week4_statefarm-sample-working.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
print (type(loaded_model))

In [ ]:
loaded_model.summary()

In [ ]:
loaded_model.count_params()

In [ ]:
loaded_model.load_weights("Week4_statefarm-sample-working_saved_weights.h5")

In [ ]:
loaded_model.count_params()

In [ ]:
type(loaded_model)

In [ ]:
for layer in loaded_model.layers:
    print (layer)
    print ("layer.get_weights:", len(layer.get_weights()))
    if len(layer.get_weights())>1:
        weights = layer.get_weights()[0]
        biases  = layer.get_weights()[1]
        print ("weights:", weights.shape)
        print ("biases:", biases.shape)

example submit format.  
csv file format.  
header row  
single decimal point probability for each category  
filename.  

img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9  
img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1  
img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1  
img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1  
